In [2]:
import os

import pandas as pd
import gspread_dataframe as gs
import pyperclip as clip # copiar o texto clipboard
from time import sleep
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as soup

import random




# Change the directory to reflect the main repository
os.chdir("C:/Users/rsilva/Google Drive/projects/programming/automation")

#import sistemas
import sei
import functions
import page
from page import *
import helpers

% reload_ext autoreload
% autoreload 2

In [ ]:
TAGS = ["@nome_interessado_maiusculas@", "@endereco_destinatario@", "@bairro_destinatario@", "@cep_destinatario@", "@cidade_destinatario@", "@validade@"]
VALUES = ["Nome", "Logradouro", ["Número", "Complemento", "Bairro"], "Cep", "Cidade", "Validade"]

In [3]:
numero = "53504.006463/2018-16"

In [4]:
gc = functions.authenticate_google("files/anatel.json")
wb = gc.open(title=numero)

In [5]:
sht = wb.get_worksheet(0)

In [6]:
df = gs.get_as_dataframe(sht, dtype=str)[:18]

In [7]:
df["Complemento"] = df["Complemento"].str.replace("nan", "")

df.drop_duplicates(inplace=True)

df.head()

,Cpf,Nome,Fistel,Validade,Filtro,End_Sede,End_Corresp,Situação,Devedor,Cpf_RF,...,Fone,Logradouro,Num,Complemento,Bairro,Cidade,UF,Cep,Endereco_Completo,Endereço_Completo
0,56380437720,ANTONIO ALVES DA MOTTA FILHO,80106277553,05/09/2017,=B2=K2,"Rua João Moreira Borges, 18, casa\n- Jardim Sa...",PRACA DA MATRIZ 12 CASA CENTRO \n CEP: 18590-...,Entidade não possui débitos,Não,563.804.377-20,...,39666687,RUA JOAO M BORGES,18,,JD STA INES I,SAO JOSE DOS CAMPOS,SP,12248-430,RUA JOAO M BORGES 18 JD STA INES I \n CEP: 12...,nan
1,33892850836,BRUNO AUGUSTO MENCK,80106228269,08/09/2017,=B3=K3,"RUA JACOB ALVES,, 40\n- CENTRO\nCEP: 18195000\...",OUTROS DO PINHEIRINHO 439 NUCLEO RES SATELITE...,Entidade devedora (Bloqueada),Sim,338.928.508-36,...,02671243,RUA JACOB ALVES,40,CASA,CENTRO,CAPELA DO ALTO,SP,18195-000,RUA JACOB ALVES 40 CASA CENTRO \n CEP: 18195-0...,nan
2,24654189807,CARLOS EDUARDO MOREIRA DE BARROS,80106091379,05/09/2017,=B4=K4,"AV RIO GRANDE, 1335\n- CENTRO\nCEP: 16370000\n...",RUA GELSUMINO LIZARDI 418 JD SAN DIEGO \n CEP...,Entidade devedora (Bloqueada),Sim,246.541.898-07,...,35410842,AVENIDA RIO GRANDE,1335,,CENTRO,PROMISSAO,SP,16370-000,AVENIDA RIO GRANDE 1335 CENTRO \n CEP: 16370-...,nan
3,18078911866,CARLOS MISTURINI JUNIOR,80106283103,06/09/2017,=B5=K5,"RUA UCHOA, 500, CASA 75\n- LOTEAMENTO SAO FRAN...",RUA OCTACILIO NOGUEIRA 41 CHAVANTES NOVO \n C...,Entidade devedora (Bloqueada),Sim,180.789.118-66,...,30161695,RODOVIA LIX DA CUNHA,7371,,JARDIM IMPERIAL,INDAIATUBA,SP,13340-200,RODOVIA LIX DA CUNHA 7371 JARDIM IMPERIAL \n ...,nan
4,9362551888,FABIO PIO DOS SANTOS,24000458108,09/09/2017,=B6=K6,"RUA WADIA JAFET ASSAD, 555, BLOCO 02 APTO 74\n...",AVENIDA 27 931 CENTRO \n CEP: 14780-340 BARR...,Atenção: Entidade devedora (Não bloqueada),Sim,093.625.518-88,...,43566193,RUA CARLOS GOMES,263,APTO 102,CENTRO,SAO BERNARDO DO CAMPO,SP,09715-130,RUA CARLOS GOMES 263 APTO 102 CENTRO \n CEP: 0...,nan


In [18]:
usr = "rsilva"
pwd = "Savorthemom3nts"
browser = webdriver.Firefox()

In [19]:
browser = sei.login_sei(browser, usr, pwd)

In [20]:
sei_ = sei.Sei(browser)

sei_.itera_processos()

In [30]:
for i in df.iterrows():
    
    dados = i[1].to_dict()    
    
    tag = sei_.atualizar_contato(dados['Nome'], dados)

In [14]:
tipo = "RC_Oficio de Cassação"

In [15]:
def string_endereço(row):
    
    s = 'À<br>' + str(row[1]["Nome"]) + '<br>' + str(row[1]["Logradouro"]) + ", " + str(row[1]["Número"]) \
         + " " + str(row[1]["Complemento"]) + " " + str(row[1]["Bairro"]) + "<br>" +  "CEP: " \
         + str(row[1]["Cep"]) +" - " + str(row[1]["Cidade"]) + "<br><br>" + "<b>FISTEL: " \
         + str(row[1]["FISTEL"]) + "</b>"
            
    return s

In [28]:
for row in df.iloc[1:,].iterrows():    
    
    dados = {'À': string_endereço(row)}    
    
    clip.copy(dados["À"])
    
    p.incluir_oficio(tipo, dados)
        
    sleep(random.randint(60,120))

In [ ]:
for p in sei_.get_processos().values():
    
    if p["aviso"] and p["tipo"] == "Outorga: Rádio do Cidadão":
        
        p = sei_.go_to_processo(p["numero"])
        
        teste = input()
        
        if teste == 'y':
            
            p.edita_marcador()
            p.edita_postit()
            
            p.get_acoes()['conc']

In [ ]:
p = sei_.go_to_processo("53504.005936/2018-50")

In [ ]:
p.edita_marcador("RC")

In [ ]:
select = Select(p.wait_for_element(helpers.Marcador.LISTA_MARCADORES))

In [34]:
windows = sei_.driver.window_handles

sei_.driver.switch_to_window(windows[0])

sei_.driver.switch_to.default_content()